In [3]:
from datasets.hand_pose_dataset import HandPoseDataset
from os.path import join, isdir
from os import listdir
import yaml
import pandas as pd

In [4]:
checkpoints_path = join("checkpoints", "results")
subject_ids = HandPoseDataset._get_subject_ids()
data = {}
metrics = []
for checkpoint_name in sorted(listdir(checkpoints_path)):
    # check if the checkpoint has complete experiments
    subjects_in_checkpoint = []
    for folder in listdir(join(checkpoints_path, checkpoint_name)):
        if folder in subject_ids and isdir(join(checkpoints_path, checkpoint_name, folder)) and "metrics.yaml" in listdir(join(checkpoints_path, checkpoint_name, folder)):
            subjects_in_checkpoint.append(folder)
    # skips empty checkpoints
    if len(subjects_in_checkpoint) <= 1:
        continue
    # loads the cfg used
    with open(join(checkpoints_path, checkpoint_name, "cfg.yaml")) as f:
        cfg = yaml.safe_load(f)
    # initialize the data dict for this checkpoint
    metrics_per_run = []
    # loops over each subject
    for subject_id in subjects_in_checkpoint:
        # load the metrics for this subject
        with open(
            join(checkpoints_path, checkpoint_name, subject_id, "metrics.yaml")
        ) as f:
            metrics_per_run.append(yaml.safe_load(f)[0])
    # compute the mean metrics for this checkpoint
    metrics_per_run = pd.DataFrame(metrics_per_run)
    metrics_per_run_mean, metrics_per_run_std = (
        metrics_per_run.mean(),
        metrics_per_run.std(),
    )
    # parses some numbers
    for col in metrics_per_run.columns:
        for df in [metrics_per_run_mean, metrics_per_run_std]:
            if col.startswith("cls_"):
                df[col] = (df[col] * 100).round(3)
            elif col.startswith("num_params"):
                df[col] = (df[col] / 1e3).astype(int)
            elif col.startswith("time_test"):
                df[col] = (df[col] * 1e3).round(3)
    metrics_per_run = (
        metrics_per_run_mean.astype(str) + " ± " + metrics_per_run_std.astype(str)
    )
    metrics_per_run["normalize_landmarks"] = cfg["normalize_landmarks"]
    metrics_per_run["image_backbone"] = cfg["image_backbone_name"]
    metrics_per_run["landmarks_backbone"] = cfg["landmarks_backbone_name"]
    # append to the metrics list
    metrics.append(metrics_per_run)

# parses the metrics into a DataFrame
metrics = pd.DataFrame(metrics)
metrics = metrics[
    [
        "normalize_landmarks",
        "landmarks_backbone",
        "image_backbone",
        "cls_acc_test",
        "cls_f1_test",
        "cls_prec_test",
        "cls_rec_test",
        "cls_loss_test",
        "num_params_test",
        "time_test",
    ]
]
metrics = metrics.rename(
    columns={
        "normalize_landmarks": "Normalize landmarks",
        "image_backbone": "Image backbone",
        "landmarks_backbone": "Landmarks backbone",
        "cls_acc_test": "Accuracy",
        "cls_f1_test": "F1",
        "cls_prec_test": "Precision",
        "cls_rec_test": "Recall",
        "cls_loss_test": "Loss",
        "num_params_test": "# Params (k)",
        "time_test": "Time (ms)",
    }
)
metrics = metrics.sort_values(by=["Landmarks backbone", "Image backbone"])
metrics = metrics.reset_index(drop=True)
print(metrics.to_latex())

\begin{tabular}{lrlllllllll}
\toprule
 & Normalize landmarks & Landmarks backbone & Image backbone & Accuracy & F1 & Precision & Recall & Loss & # Params (k) & Time (ms) \\
\midrule
0 & True & mlp & NaN & 90.887 ± 3.419 & 90.599 ± 3.59 & 91.689 ± 3.067 & 90.887 ± 3.419 & 81.822 ± 6.904 & 4177.0 ± 0.0 & 0.381 ± 0.017 \\
1 & False & mlp & NaN & 79.678 ± 5.33 & 79.661 ± 5.205 & 83.249 ± 4.14 & 79.678 ± 5.33 & 115.922 ± 11.002 & 4177.0 ± 0.0 & 0.376 ± 0.025 \\
\bottomrule
\end{tabular}

